In [71]:
%pylab inline
from bs4 import BeautifulSoup
import requests
import csv
import time
import pandas as pd
import re

Populating the interactive namespace from numpy and matplotlib


In [10]:
url = "www.scm.com/doc/ReaxFF/Included_Forcefields_Trunk.html"
page = requests.get("http://" +url)
soup = BeautifulSoup(page.text, 'html.parser')

In [19]:
models = soup.find("dl", class_ = "docutils")
all_models = models.text

In [132]:
def models_tokenizer(text):
    models_df = pd.DataFrame(columns = ["File", "Atoms", "Molecule", "Branch", "Description"])
    models_list = all_models.split("\n\n\n")
    for model in models_list:
        model_content = [x for x in model.splitlines() if x != '']
        
        # first line of the description
        model_header = model_content[0].split()
        file = model_header[0][:-1]
        atoms = ','.join([x for x in re.split("\/|\(|\)", model_header[1]) if x != ''])
        molecule = ' '.join(model_header[2:])
        
        # second line refers to the actual paper
        paper = model_content[1]
        
        # last line refers to the branch, but it doe snot always exist
        if "Branch" in model_content[-1]:
            desc = ' '.join(model_content[2:-1])
            branch = model_content[-1].split()[-1][:-1]
        else:
            desc = ' '.join(model_content[2:])
            branch = ''
        
        # creating a series from pandas and then concatenating

        
        model_series = pd.Series({"File":file, "Atoms":atoms, "Molecule":molecule, "Branch":branch, "Description": desc})
        models_df = models_df.append(model_series, ignore_index = True)
    
    return models_df

In [153]:
def find_file(atoms_to_find, df):
    # e.g. atoms_to_find = "C O H"
    atoms = atoms_to_find.split()
    atoms_index = [False] * len(df)
    for i in range(len(df)):
        for atom in atoms:
            if atom in df.Atoms[i]:
                atoms_index[i] = True
    return atoms_index

In [134]:
modelsDF = models_tokenizer(all_models)
modelsDF.head()

,File,Atoms,Molecule,Branch,Description
0,AB.ff,"H,O,N,B",Ammonia Borane,combustion,QM data were generated describing the single a...
1,AuCSOH.ff,"Au,C,S,O,H",,water,The original Au-Au parameters were extended by...
2,CHO.ff,"C,H,O",Hydrocarbon oxidation,combustion,To obtain the H/C/O compound data required to ...
3,HCONSB.ff,"H,C,O,N,S,B",,combustion,The parameters in this forcefield were extende...
4,CuCl-H2O.ff,"Cu,Cl,H,O",,water,This forcefield is an extension of: A.C.T. van...


In [157]:
modelsDF[find_file('Fe', modelsDF)]

,File,Atoms,Molecule,Branch,Description
5,FeOCHCl.ff,"Fe,O,C,H,Cl",,water,The Cl parameters where published by: O. Raham...
15,CaSiAlO.ff,"C,H,O,Fe,Cl,Si,Al,Ca",,water,
17,CHOFeAlNiCuS.ff,"C,H,O,Fe,Al,Ni,Cu,S",,water,C/O/H parameters only. The Cu/Fe/Al/Ni paramet...
27,CHOFe.ff,"C,H,O,Fe,Cl,Si,Al",,water,only the parameters for Fe (and crossterms) di...
28,CHOAlSi.ff,"C,H,O,Fe,Cl,Si,Al",,water,only the parameters for Fe (and crossterms) di...
50,CHOFeAlNiCuSCr.ff,"C,H,O,Fe,Al,Ni,Cu,S,Cr",,water,Forcefield optimized for Fe/Cr/O/S
51,CHOFeAlNiCuSCr_v2.ff,"C,H,O,Fe,Al,Ni,Cu,S,Cr",Ni-O-vacancy,water,"Forcefield optimized for Ni/O, trained with a ..."
52,CHOFeAlNiCuSCr_v3.ff,"C,H,O,Fe,Al,Ni,Cu,S,Cr",,water,Combination of the C.ff (condensed carbon) for...
70,CHOFeAlNiCuSCrSiGe.ff,"C,H,O,Fe,Al,Ni,Cu,S,Cr,Si,Ge",,,Used for studying the ALD process of Al2O3 fro...


In [165]:
modelsDF.iloc[17].Description

'C/O/H parameters only. The Cu/Fe/Al/Ni parameters are from: Y.K. Shin, H. Kwak, C. Zou, A.V. Vasenkov, and A.C.T. van Duin Development and Validation of a ReaxFF Reactive Force Field for Fe/Al/Ni Alloys: Molecular Dynamics Study of Elastic Constants, Diffusion, and Segregation J. Phys. Chem. A, 2012, 116 (49), pp 12163–12174 http://dx.doi.org/10.1021/jp308507x Not all cross-terms between the two forcefield files are defined, which might cause problems if the system has (for example) C-Cu interactions.'